# Day 1

## Imports and data loading

In [1]:
from utils import get_input, load_data
day = 8

In [5]:
get_input(day)

Data saved


In [2]:
data = load_data(day, list_type="line", number=False)
test_data = [
    "be cfbegad cbdgef fgaecd cgeb fdcge agebfd fecdb fabcd edb | fdgacbe cefdb cefbgd gcbe",
    "edbfga begcd cbg gc gcadebf fbgde acbgfd abcde gfcbed gfec | fcgedb cgb dgebacf gc",
    "fgaebd cg bdaec gdafb agbcfd gdcbef bgcad gfac gcb cdgabef | cg cg fdcagb cbg",
    "fbegcd cbd adcefb dageb afcb bc aefdc ecdab fgdeca fcdbega | efabcd cedba gadfec cb",
    "aecbfdg fbg gf bafeg dbefa fcge gcbea fcaegb dgceab fcbdga | gecf egdcabf bgf bfgea",
    "fgeab ca afcebg bdacfeg cfaedg gcfdb baec bfadeg bafgc acf | gebdcfa ecba ca fadegcb",
    "dbcfg fgd bdegcaf fgec aegbdf ecdfab fbedc dacgb gdcebf gf | cefg dcbef fcge gbcadfe",
    "bdfegc cbegaf gecbf dfcage bdacg ed bedf ced adcbefg gebcd | ed bcgafe cdgba cbgef",
    "egadfb cdbfeg cegd fecab cgb gbdefca cg fgcdab egfdb bfceg | gbdfcae bgc cg cgb",
    "gcafb gcf dcaebfg ecagb gf abcdeg gaef cafbge fdbac fegbdc | fgae cfgab fg bagce",
]
test_answer_1 = 26
test_answer_2 = None

## Prep

In [3]:
num_to_segment = {
    0: 0,
    1: 2,
    2: 5,
    3: 5,
    4: 4,
    5: 5,
    6: 6,
    7: 3,
    8: 7,
    9: 6,
}
segment_to_num = {x: [] for x in range(10)}

In [4]:
def split_data(inputs) -> tuple:
    divided = list(zip(*[line.partition(" | ") for line in inputs]))
    return divided[0], divided[2]

def count_digits(after) -> list:
    lengths = [[len(c.strip())for c in codes.split()] for codes in after]
    return lengths

def count_uniques(counts) -> int:
    total = 0
    for line in counts:
        for val in line:
            if val in [2, 4, 3, 7]:
                total +=1
    return total


def part_one_answer(data) -> int:
    counts = count_digits(data)
    return count_uniques(counts)


## Part one

In [5]:
divided_test = split_data(test_data)
divided = split_data(data)

assert part_one_answer(divided_test[1]) == test_answer_1
part_one_answer(divided[1])

512

## Part two

In [35]:
# a
#b c
# d
#e f
# g

# give each of a to g a set of possible translations, then gradually reduce them?
LETTERS = {"a", "b", "c", "d", "e", "f", "g"}
DISPLAYS = {
    ("a", "b", "c", "e", "f", "g"): 0,
    ("c", "f"): 1,
    ("a", "c", "d", "e", "g"): 2,
    ("a", "c", "d", "f", "g"): 3,
    ("b", "c", "d", "f"): 4,
    ("a", "b", "d", "f", "g"): 5,
    ("a", "b", "d", "e", "f", "g"): 6,
    ("a", "c", "f"): 7,
    ("a", "b", "c", "d", "e", "f", "g"): 2,
    ("a", "b", "c", "d", "f", "g"): 9,
}

class LetterBoard():
    def __init__(self, line):
        divided = line.partition(" | ")
        self.codes = [set(i.strip()) for i in divided[0].split()]
        self.output = [set(d) for d in divided[2].split()]
        self.possible_letters = {letter: LETTERS.copy() for letter in LETTERS}
        self.translations = {}

    def limit_letters(self, letters_used: set, intended_letters: str):
        for letter in self.possible_letters:
            if letter in letters_used:
                self.possible_letters[letter] &= set(intended_letters)
            else:
                self.possible_letters[letter] -= set(intended_letters)

    def untangle(self):
        """Work out the actual letter represented by each of the used letters"""
        for code in self.codes:
            if len(code) == 2:
                self.limit_letters(code, "cf")
            elif len(code) == 3:
                self.limit_letters(code, "acf")
            elif len(code) == 4:
                self.limit_letters(code, "bcdf")

    def identify_numbers(self):
        for code in self.output:
            translated = set()
            for c in code:
                translated.update(c)
            hashable = tuple(sorted(translated))
            number = DISPLAYS.get(hashable, "unknown")
            if number == "unknown":
                print(f"Can't convert {code} to integer")
                print(self.possible_letters)
                print()
            return number
        

    def convert_to_number(self):
        """Check all the letters have only one possible translation."""
        self.untangle()
        return self.identify_numbers()

In [36]:
total = 0

for test in test_data:
    board = LetterBoard(test)
    total += board.convert_to_number()

total

Can't convert {'b', 'g', 'd', 'c', 'e', 'f'} to integer
{'f': {'b', 'd'}, 'e': {'b', 'd'}, 'c': {'f', 'c'}, 'b': {'a'}, 'a': {'e', 'g'}, 'g': {'f', 'c'}, 'd': {'e', 'g'}}



TypeError: unsupported operand type(s) for +=: 'int' and 'str'